In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrames").getOrCreate()

In [ ]:
import os
os.getcwd()

In [ ]:
spark

In [7]:
sc = spark.sparkContext

In [9]:
from pyspark.sql.window import Window
import pyspark.sql.functions as psf
from pyspark.sql.functions import *
from collections import defaultdict
from pyspark.sql import Row

# Graph like logic in spark SQL

In [10]:
#validate
l = [(1,1),
    (2,2),
    (3,1),
    (4,4),
    (5,1),
    (6,6),
    (7,7),
    (8,8),
    (9,3),
    (9,6),
    (10,1),
    (11,11),
    (12,9),
    (13,13),
    (14,14),
    (15,9),
    (16,16),
    (17,17),
    (18,15),
    (18,16),
    (19,15),
    (19,13),
    (20,20),
    (51,51),
    (55,51),
    (57,57),
    (59,57),
    (61,55),
    (61,59)]
rdd = sc.parallelize(l)
ucid_mpp_acc = rdd.map(lambda x: Row(u=x[0],p=x[1]))
np = spark.createDataFrame(ucid_mpp_acc)

np.show(50)

np2 = np.withColumnRenamed('p','p2').withColumnRenamed('u','u2')
np2.printSchema()

+---+---+
|  u|  p|
+---+---+
|  1|  1|
|  2|  2|
|  3|  1|
|  4|  4|
|  5|  1|
|  6|  6|
|  7|  7|
|  8|  8|
|  9|  3|
|  9|  6|
| 10|  1|
| 11| 11|
| 12|  9|
| 13| 13|
| 14| 14|
| 15|  9|
| 16| 16|
| 17| 17|
| 18| 15|
| 18| 16|
| 19| 15|
| 19| 13|
| 20| 20|
| 51| 51|
| 55| 51|
| 57| 57|
| 59| 57|
| 61| 55|
| 61| 59|
+---+---+

root
 |-- u2: long (nullable = true)
 |-- p2: long (nullable = true)



In [ ]:
from collections import defaultdict

loop = 1
debug = 0
happy = 0
happy_mp = 1
happy_gp = 1
infinit_loop_exit = 100
df_dict = defaultdict(list)
df_dict[1].append(np)
df_dict[1].append(np2)
##print('Start time:', str(datetime.datetime.now()))
while not happy: 
    print('doing the {0} check'.format(loop))
    print('checking for multiple parents.')
    #mp = df_dict[loop][0].join(df_dict[loop][0].groupBy("u").agg(countDistinct("p").alias('count_{0}'.format(loop))), 'u')\
    #            .filter('count_{0} > 1'.format(loop))        
    mp = df_dict[loop][0].join(df_dict[loop][0].groupBy("u").agg(countDistinct("p").alias('count')), 'u')\
               .filter('count > 1')
    #mp.show()    
    print('rows with multiple parents issues:',mp.count(), 'check_{0}'.format(loop))
    if mp.count() > 0:
        print('adding the parents rel.')
        happy_mp = 0
        wd = Window.partitionBy('w').orderBy('u')
        mp_rel = mp.withColumnRenamed('u','w').withColumnRenamed('p','u').withColumn('p', psf.min('u').over(wd))
        new_rel = mp_rel.filter('u != p').select('u', 'p').distinct()
        if debug == 1:
            new_rel.show()
        np_a = df_dict[loop][0].select('u', 'p').union(new_rel).distinct()
        if debug == 1:
            #np.sort('u').show(50)
            np_a.show(50)
        np = np_a.join(np_a.groupBy("u").agg(countDistinct("p").alias('count_{0}'.format(loop))), 'u')\
                               .filter('(count_{0} > 1 and u!=p) or count_{0} = 1'.format(loop))
        if debug == 1:
            # np.sort('u').show()
            np.show(50)
        df_dict[loop+1].append(np)
        #generate np2 in case we down't go into grandpa issues
        df_dict[loop+1].append(np.withColumnRenamed('p','p2').withColumnRenamed('u','u2'))
        #overwrite np and np2 for loop in case we go into grandpa issues
        df_dict[loop][0] = np
        df_dict[loop][1] = np.withColumnRenamed('p','p2').withColumnRenamed('u','u2')
        
        
        # cleanup
        mp_rel.unpersist()
        new_rel.unpersist()
        np_a.unpersist()
        
    else:
        happy_mp = 1


   
    print('checking for grandparents.')
    chk = df_dict[loop][0].join(df_dict[loop][1], df_dict[loop][0].p==df_dict[loop][1].u2)\
            .filter(df_dict[loop][0].p!=df_dict[loop][1].p2)
    print('rows with grandpa issues:',chk.count())
    
    # new parent df
    if chk.count() > 0 :
        print('adding the grandparents.')
        happy_gp = 0
        np_temp = df_dict[loop][0].join(df_dict[loop][1], df_dict[loop][0].p==df_dict[loop][1].u2)\
                    .select(df_dict[loop][0].u,df_dict[loop][1].p2)\
                    .withColumnRenamed('u','u').withColumnRenamed('p2','p').distinct()
        np = np_temp
        if debug:
            print('gp resulting np:')
            np.sort('u', 'p').show(60)
        np2 = np.withColumnRenamed('p','p2').withColumnRenamed('u','u2')
        if df_dict[loop+1][0]:
            df_dict[loop+1][0] = np
            df_dict[loop+1][1] = np2
        else:
            df_dict[loop+1].append(np)
            df_dict[loop+1].append(np2)
            
        # cleanup
        np_temp.unpersist()
        
    else:
        happy_gp = 1
    
    print('Loop {} happy_gp:'.format(loop+1),happy_gp)
    print('Loop {} happy_mp:'.format(loop+1), happy_mp)
    
    if happy_gp and happy_mp:
        happy = 1
    
    if not happy:
        df_dict[loop+1][0] = df_dict[loop+1][0].cache()
        df_dict[loop+1][1] = df_dict[loop+1][1].cache()
        df_dict[loop][0].unpersist()
        df_dict[loop][1].unpersist()
    
    
    if infinit_loop_exit < loop:
        happy = 1
    
    loop = loop + 1
    
print('End time:', str(datetime.datetime.now()))

print('The dataframe with the right Parent - - Child relationship.')
df_dict[loop-1][0].sort('u').show(50)

In [ ]:
np.sort('u').show(50)

In [ ]:
spark.stop()

# Graph

#### the next cell needs to be executed with a fresh kernel in order for the library to be loaded in spark
if we were to use spark-skell we would add --packages graphframes:graphframes:0.8.1-spark3.0-s_2.12

multiple spark packages can be found at https://spark-packages.org/

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Graph")\
    .config('spark.jars.packages', 'graphframes:graphframes:0.8.1-spark3.0-s_2.12')\
    .getOrCreate()

In [2]:
spark

In [3]:
sqlContext = spark

In [4]:


# Create a Vertex DataFrame with unique ID column "id"
v = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = sqlContext.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
], ["src", "dst", "relationship"])
# Create a GraphFrame
from graphframes import *
g = GraphFrame(v, e)

# Query: Get in-degree of each vertex.
g.inDegrees.show()

# Query: Count the number of "follow" connections in the graph.
g.edges.filter("relationship = 'follow'").count()

# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()



+---+--------+
| id|inDegree|
+---+--------+
|  c|       1|
|  b|       2|
+---+--------+

+---+------------------+
| id|          pagerank|
+---+------------------+
|  b|1.0905890109440908|
|  a|              0.01|
|  c|1.8994109890559092|
+---+------------------+



#### lets try to implement the same logic as in the unique id example from the powerpoint


In [5]:
# create a list of nodes/vertexes
v = spark.createDataFrame(
    [
        (1, "A"),
        (2, "B"),
        (3, "C"),
        (4, "D"),
        (5, "E"),
        (6, "F"),
        (7, "G"),
        (8, "H"),
        (9, "I"),
        (10,"J"),
        (11,"K")
        
    ],
        ['id', 'name']
)

In [6]:
# and a list of relationships/edges
e = spark.createDataFrame(
    [
        (1, 1, "A"),
        (2, 2, "B"),
        (3, 3, "C"),
        (4, 4, "D"),
        (5, 5, "E"),
        (6, 2, "F"),
        (6, 3, "F"),
        (7, 4, "G"),
        (7, 5, "G"),
        (8, 6, "H"),
        (9, 7, "I"),
        (9, 8, "I"),
        (10,10, "J"),
        (11,10, "K")
        
    ],
        ['src', 'dst', "source"]
)

In [7]:
g = GraphFrame(v, e)

In [8]:
# we can use triplets to show the graph as a DF
# sorting by src we can see that 6,7,9 have multiple parents
g.triplets.sort('src').show()

+-------+-----------+-------+
|    src|       edge|    dst|
+-------+-----------+-------+
| [1, A]|  [1, 1, A]| [1, A]|
| [2, B]|  [2, 2, B]| [2, B]|
| [3, C]|  [3, 3, C]| [3, C]|
| [4, D]|  [4, 4, D]| [4, D]|
| [5, E]|  [5, 5, E]| [5, E]|
| [6, F]|  [6, 3, F]| [3, C]|
| [6, F]|  [6, 2, F]| [2, B]|
| [7, G]|  [7, 5, G]| [5, E]|
| [7, G]|  [7, 4, G]| [4, D]|
| [8, H]|  [8, 6, H]| [6, F]|
| [9, I]|  [9, 7, I]| [7, G]|
| [9, I]|  [9, 8, I]| [8, H]|
|[10, J]|[10, 10, J]|[10, J]|
|[11, K]|[11, 10, K]|[10, J]|
+-------+-----------+-------+



In [10]:
# graph require a checkpoint directory
# a tmp folder which is used for dumping temporary data
sc = spark.sparkContext
sc.setCheckpointDir('graphframes_cps')

In [11]:
# we can get the unique groups
unique_groups = g.connectedComponents()

In [12]:
type(unique_groups)

pyspark.sql.dataframe.DataFrame

In [13]:
unique_groups.show()

+---+----+---------+
| id|name|component|
+---+----+---------+
|  1|   A|        1|
|  2|   B|        2|
|  3|   C|        2|
|  4|   D|        2|
|  5|   E|        2|
|  6|   F|        2|
|  7|   G|        2|
|  8|   H|        2|
|  9|   I|        2|
| 10|   J|       10|
| 11|   K|       10|
+---+----+---------+



just for fun lets see how long each version works

In [47]:
%%time
unique_groups2 = g.connectedComponents()
unique_groups2.show()

+---+----+---------+
| id|name|component|
+---+----+---------+
|  1|   A|        1|
|  2|   B|        2|
|  3|   C|        2|
|  4|   D|        2|
|  5|   E|        2|
|  6|   F|        2|
|  7|   G|        2|
|  8|   H|        2|
|  9|   I|        2|
+---+----+---------+

CPU times: user 233 ms, sys: 117 ms, total: 351 ms
Wall time: 27.4 s


In [53]:
#validate
l = [
    (1, 1),
    (2, 2),
    (3, 3),
    (4, 4),
    (5, 5),
    (6, 2),
    (6, 3),
    (7, 4),
    (7, 5),
    (8, 6),
    (9, 7),
    (9, 8)]
rdd = sc.parallelize(l)
ucid_mpp_acc = rdd.map(lambda x: Row(u=x[0],p=x[1]))
np = spark.createDataFrame(ucid_mpp_acc)

np.show(50)

np2 = np.withColumnRenamed('p','p2').withColumnRenamed('u','u2')
# np2.printSchema()

+---+---+
|  u|  p|
+---+---+
|  1|  1|
|  2|  2|
|  3|  3|
|  4|  4|
|  5|  5|
|  6|  2|
|  6|  3|
|  7|  4|
|  7|  5|
|  8|  6|
|  9|  7|
|  9|  8|
+---+---+



In [54]:
%%time
from collections import defaultdict

loop = 1
debug = 0
happy = 0
happy_mp = 1
happy_gp = 1
infinit_loop_exit = 100
df_dict = defaultdict(list)
df_dict[1].append(np)
df_dict[1].append(np2)
##print('Start time:', str(datetime.datetime.now()))
while not happy: 
    print('doing the {0} check'.format(loop))
    print('checking for multiple parents.')
    #mp = df_dict[loop][0].join(df_dict[loop][0].groupBy("u").agg(countDistinct("p").alias('count_{0}'.format(loop))), 'u')\
    #            .filter('count_{0} > 1'.format(loop))        
    mp = df_dict[loop][0].join(df_dict[loop][0].groupBy("u").agg(countDistinct("p").alias('count')), 'u')\
               .filter('count > 1')
    #mp.show()    
    print('rows with multiple parents issues:',mp.count(), 'check_{0}'.format(loop))
    if mp.count() > 0:
        print('adding the parents rel.')
        happy_mp = 0
        wd = Window.partitionBy('w').orderBy('u')
        mp_rel = mp.withColumnRenamed('u','w').withColumnRenamed('p','u').withColumn('p', psf.min('u').over(wd))
        new_rel = mp_rel.filter('u != p').select('u', 'p').distinct()
        if debug == 1:
            new_rel.show()
        np_a = df_dict[loop][0].select('u', 'p').union(new_rel).distinct()
        if debug == 1:
            #np.sort('u').show(50)
            np_a.show(50)
        np = np_a.join(np_a.groupBy("u").agg(countDistinct("p").alias('count_{0}'.format(loop))), 'u')\
                               .filter('(count_{0} > 1 and u!=p) or count_{0} = 1'.format(loop))
        if debug == 1:
            # np.sort('u').show()
            np.show(50)
        df_dict[loop+1].append(np)
        #generate np2 in case we down't go into grandpa issues
        df_dict[loop+1].append(np.withColumnRenamed('p','p2').withColumnRenamed('u','u2'))
        #overwrite np and np2 for loop in case we go into grandpa issues
        df_dict[loop][0] = np
        df_dict[loop][1] = np.withColumnRenamed('p','p2').withColumnRenamed('u','u2')
        
        
        # cleanup
        mp_rel.unpersist()
        new_rel.unpersist()
        np_a.unpersist()
        
    else:
        happy_mp = 1


   
    print('checking for grandparents.')
    chk = df_dict[loop][0].join(df_dict[loop][1], df_dict[loop][0].p==df_dict[loop][1].u2)\
            .filter(df_dict[loop][0].p!=df_dict[loop][1].p2)
    print('rows with grandpa issues:',chk.count())
    
    # new parent df
    if chk.count() > 0 :
        print('adding the grandparents.')
        happy_gp = 0
        np_temp = df_dict[loop][0].join(df_dict[loop][1], df_dict[loop][0].p==df_dict[loop][1].u2)\
                    .select(df_dict[loop][0].u,df_dict[loop][1].p2)\
                    .withColumnRenamed('u','u').withColumnRenamed('p2','p').distinct()
        np = np_temp
        if debug:
            print('gp resulting np:')
            np.sort('u', 'p').show(60)
        np2 = np.withColumnRenamed('p','p2').withColumnRenamed('u','u2')
        if df_dict[loop+1][0]:
            df_dict[loop+1][0] = np
            df_dict[loop+1][1] = np2
        else:
            df_dict[loop+1].append(np)
            df_dict[loop+1].append(np2)
            
        # cleanup
        np_temp.unpersist()
        
    else:
        happy_gp = 1
    
    print('Loop {} happy_gp:'.format(loop+1),happy_gp)
    print('Loop {} happy_mp:'.format(loop+1), happy_mp)
    
    if happy_gp and happy_mp:
        happy = 1
    
    if not happy:
        df_dict[loop+1][0] = df_dict[loop+1][0].cache()
        df_dict[loop+1][1] = df_dict[loop+1][1].cache()
        df_dict[loop][0].unpersist()
        df_dict[loop][1].unpersist()
    
    
    if infinit_loop_exit < loop:
        happy = 1
    
    loop = loop + 1
    
print('The dataframe with the right Parent - - Child relationship.')
df_dict[loop-1][0].sort('u').show(50)

doing the 1 check
checking for multiple parents.
rows with multiple parents issues: 6 check_1
adding the parents rel.
checking for grandparents.
rows with grandpa issues: 10
adding the grandparents.
Loop 2 happy_gp: 0
Loop 2 happy_mp: 0
doing the 2 check
checking for multiple parents.
rows with multiple parents issues: 8 check_2
adding the parents rel.
checking for grandparents.
rows with grandpa issues: 13
adding the grandparents.
Loop 3 happy_gp: 0
Loop 3 happy_mp: 0
doing the 3 check
checking for multiple parents.
rows with multiple parents issues: 4 check_3
adding the parents rel.
checking for grandparents.
rows with grandpa issues: 2
adding the grandparents.
Loop 4 happy_gp: 0
Loop 4 happy_mp: 0
doing the 4 check
checking for multiple parents.
rows with multiple parents issues: 0 check_4
checking for grandparents.
rows with grandpa issues: 0
Loop 5 happy_gp: 1
Loop 5 happy_mp: 1
The dataframe with the right Parent - - Child relationship.
+---+---+
|  u|  p|
+---+---+
|  1|  1|
|  